### 1. Mount Google Drive
Mounts Google Drive to access the processed Computer Vision (CV) dataset.

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


### 2. Initialize Qdrant Client
Initializes the connection to the Qdrant Cloud vector database. This client will be used to create the `cv_collection` and upload vectors.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models as rest

qdrant_client = QdrantClient(
     url="Enter Your api Url****",
     api_key="Enter your Api Key**"
)


### 3. Inspect Processed Data
Reads and prints the first line of the processed CV dataset (`cv.jsonl`) to verify the data format before indexing.

In [9]:
import json

path = "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/cv.jsonl"

with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(json.loads(line))
        break


{'openalex_id': 'https://openalex.org/W4300424419', 'doi': 'https://doi.org/10.4230/lipics.giscience.2025.18', 'title': 'U-Prithvi: Integrating a Foundation Model and U-Net for Enhanced Flood Inundation Mapping', 'publication_year': 2025, 'publication_date': '2025-01-01', 'authors': [{'author_id': 'https://openalex.org/A5045981348', 'name': 'Olaf Ronneberger'}, {'author_id': 'https://openalex.org/A5041583196', 'name': 'Philipp Fischer'}, {'author_id': 'https://openalex.org/A5070290355', 'name': 'Thomas Brox'}], 'concepts': [{'id': 'https://openalex.org/C41008148', 'name': 'Computer science'}, {'id': 'https://openalex.org/C154945302', 'name': 'Artificial intelligence'}, {'id': 'https://openalex.org/C89600930', 'name': 'Segmentation'}, {'id': 'https://openalex.org/C774472', 'name': 'Margin (machine learning)'}, {'id': 'https://openalex.org/C102392041', 'name': 'Sliding window protocol'}, {'id': 'https://openalex.org/C2779343474', 'name': 'Context (archaeology)'}, {'id': 'https://openalex

### 4. Full Indexing Pipeline (CV Domain)
This is the main ingestion script for the Computer Vision domain:
1.  **Config**: Sets the collection name (`cv_collection`), batch size, and vector size.
2.  **Setup**: Creates the Qdrant collection if it doesn't exist.
3.  **Ingest**:
    *   Reads the `cv.jsonl` file.
    *   Generates embeddings using `all-MiniLM-L6-v2`.
    *   Constructs the payload with metadata.
    *   Uploads points in batches to Qdrant.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
import json
from tqdm import tqdm

# ---------------- CONFIG ----------------
COLLECTION = "cv_collection"   # ✅ NEW COLLECTION
BATCH_SIZE = 64             # safer for cloud
VECTOR_SIZE = 384

# ---------------- MODEL -----------------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ---------------- QDRANT ----------------
client = QdrantClient(
    url="Enter Your api Url****",
     api_key="Enter your Api Key**"
)

# ---------------- COLLECTION SETUP ----------------
if not client.collection_exists(COLLECTION):
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config=VectorParams(
            size=VECTOR_SIZE,
            distance=Distance.COSINE
        )
    )

# ---------------- INGEST ----------------
points = []
idx = 0

with open(
    "/content/drive/MyDrive/OpenAlex_CS_2025_Data/processed/cv.jsonl",
    "r",
    encoding="utf-8"
) as f:

    for line in tqdm(f, desc="Indexing AI Papers"):
        paper = json.loads(line)

        text = f"{paper.get('title','')} {paper.get('abstract','')}".strip()
        if len(text) < 30:
            continue

        vector = model.encode(text).tolist()

        payload = {
            "openalex_id": paper.get("openalex_id"),
            "doi": paper.get("doi"),
            "title": paper.get("title"),
            "abstract": paper.get("abstract"),
            "publication_year": paper.get("publication_year"),
            "publication_date": paper.get("publication_date"),
            "venue": paper.get("venue"),
            "citation_count": paper.get("citation_count"),
            "is_open_access": paper.get("is_open_access"),
            "oa_status": paper.get("oa_status"),
            "url": paper.get("url"),

            # 🔥 STRUCTURED FIELDS
            "authors": [
                {
                    "id": a.get("author_id"),
                    "name": a.get("name")
                } for a in paper.get("authors", [])
            ],

            "concepts": [
                {
                    "id": c.get("id"),
                    "name": c.get("name")
                } for c in paper.get("concepts", [])
            ]
        }

        points.append(
            PointStruct(
                id=idx,
                vector=vector,
                payload=payload
            )
        )
        idx += 1

        if len(points) == BATCH_SIZE:
            client.upsert(
                collection_name=COLLECTION,
                points=points,
                wait=False  # ✅ avoids timeout
            )
            points = []

# Final flush
if points:
    client.upsert(
        collection_name=COLLECTION,
        points=points,
        wait=False
    )

print("✅ FULL CV COLLECTION INDEXED SUCCESSFULLY")

Indexing AI Papers: 19221it [06:45, 47.43it/s]


✅ FULL CV COLLECTION INDEXED SUCCESSFULLY


### 5. Test Retrieval (CV Domain)
Performs a sanity check on the indexed `cv_collection`:
1.  Encodes a test query ("What are the open research gaps in CV?").
2.  Searches the collection for the top 5 nearest neighbors.
3.  Prints the detailed results to verify retrieval quality.

In [ ]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

client = QdrantClient(
    url="Enter Your api Url****",
     api_key="Enter your Api Key**"
)

query = "What are the open research gaps in CV?"
vector = model.encode(query).tolist()

results = client.query_points(
    collection_name="cv_collection" ,
    query=vector,
    limit=5,
    with_payload=True
)

print(f"Returned points: {len(results.points)}")

for i, hit in enumerate(results.points, start=1):
    p = hit.payload

    print(f"\n--- Result {i} ---")
    print("Score:", round(hit.score, 4))
    print("Title:", p.get("title"))
    print("DOI:", p.get("doi"))
    print("Year:", p.get("publication_year"))
    print("Venue:", p.get("venue"))
    print("Citations:", p.get("citation_count"))
    print("Open Access:", p.get("is_open_access"), "| Status:", p.get("oa_status"))
    print("URL:", p.get("url"))

    # ✅ SAFE abstract handling
    abstract = p.get("abstract")
    if abstract:
        print("Abstract:", abstract[:600])
    else:
        print("Abstract: ❌ Not available")

    # ✅ Authors
    authors = p.get("authors", [])
    if authors:
        print("Authors:", ", ".join(a["name"] for a in authors[:5]))
    else:
        print("Authors: ❌ Not available")

    # ✅ Concepts
    concepts = p.get("concepts", [])
    if concepts:
        print("Concepts:", ", ".join(c["name"] for c in concepts[:6]))
    else:
        print("Concepts: ❌ Not available")

Returned points: 5

--- Result 1 ---
Score: 0.3224
Title: Sampling Thoughtfully in Qualitative Research: Hallmarks of Rigour in Health Professions Education
DOI: https://doi.org/10.1111/tct.70145
Year: 2025
Venue: The Clinical Teacher
Citations: 3
Open Access: True | Status: hybrid
URL: https://openalex.org/W4412182847
Abstract: ❌ Not available
Authors: Jerusalem Merkebu, Jamie Geringer, Anita Samuel
Concepts: Rigour, Sampling (signal processing), Qualitative research, Psychology, Medical education, Engineering ethics

--- Result 2 ---
Score: 0.3129
Title: Exploring Sampling Strategies to Maximize Qualitative Research Studies in Adult Education
DOI: https://doi.org/10.1177/10451595251349183
Year: 2025
Venue: Adult Learning
Citations: 1
Open Access: False | Status: closed
URL: https://openalex.org/W4411049963
Abstract: ❌ Not available
Authors: Anita Samuel, Jerusalem Merkebu
Concepts: Qualitative research, Sampling (signal processing), Adult education, Psychology, Mathematics education,